In [ ]:
# Install qubex library if not installed
# !pip install git+https://github.com/amachino/qubex.git

In [ ]:
import numpy as np
import qubex as qx

In [ ]:
# Create a flat-top pulse
flattop = qx.pulse.FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)

# Plot the pulse
flattop.plot()

# Print the pulse values
print(flattop.values)

In [ ]:
# Scale the amplitude of the pulse
flattop.scaled(1.5).plot()

# Shift the phase of the pulse
flattop.shifted(np.pi).plot()

# Repeat the pulse
flattop.repeated(3).plot()

In [ ]:
# Create a Gaussian pulse
gaussian = qx.pulse.Gaussian(
    duration=60,
    amplitude=0.5,
    sigma=10,
)

# Create a pulse array
arr = qx.PulseArray(
    [
        gaussian.shifted(np.pi / 2),
        qx.VirtualZ(np.pi / 2),
        gaussian.shifted(-np.pi / 2),
    ]
)

# Plot the logical pulse array
arr.plot(show_physical_pulse=False)

# Plot the physical pulse array
arr.plot(show_physical_pulse=True)

# Print the pulse array values
print(arr.values)

In [ ]:
# Create an arbitrary pulse
iq_array = [
    0.1 + 0.2j,
    0.2 + 0.3j,
    0.3 + 0.4j,
    0.4 + 0.6j,
    0.5 + 0.6j,
]
arbit = qx.Pulse(iq_array)

# Plot the pulse
arbit.plot()

# Plot the reverse of the pulse
arbit.reversed().plot()

In [ ]:
# Create a pulse schedule for the channels
with qx.PulseSchedule() as ps1:
    ps1.add("Q00", flattop.repeated(5))
    ps1.add("Q01", arr.scaled(2))
    ps1.barrier()
    ps1.add("Q00", arbit.repeated(3))
    ps1.barrier(["Q00", "Q01"])
    ps1.add("Q01", qx.pulse.FlatTop(duration=100, amplitude=1, tau=10))
    ps1.add("Q02", qx.PulseArray([gaussian, qx.VirtualZ(np.pi / 2), gaussian]))

# Plot the pulse schedule
ps1.plot()

# Get the PulseArray dictionary of the channels
ps1.get_sequences()

In [ ]:
# Get the sampled sequences of the channels
ps1.get_sampled_sequences()

In [ ]:
# Create a pulse schedule using the previous pulse schedule

with qx.PulseSchedule() as ps2:
    ps2.call(ps1)
    ps2.barrier()
    ps2.add("Q00", qx.Blank(100))
    ps2.barrier()
    ps2.call(ps1.reversed())

ps2.plot()

In [ ]:
# Cross-resonance sequence

x180 = qx.pulse.Drag(duration=16, amplitude=0.02, beta=0.1)

zx90 = qx.pulse.CrossResonance(
    control_qubit="Q00",
    target_qubit="Q01",
    cr_amplitude=1.0,
    cr_duration=200,
    cr_ramptime=30,
    cr_phase=0.0,
    cancel_amplitude=0.01,
    cancel_phase=np.pi / 6,
    echo=True,
    pi_pulse=x180,
)

zx90.plot()

In [ ]:
# CNOT sequence

x90 = qx.pulse.Drag(duration=16, amplitude=0.01, beta=0.1)
x90m = x90.scaled(-1)
z90m = qx.VirtualZ(-np.pi / 2)

with qx.PulseSchedule() as cnot:
    cnot.call(zx90)
    cnot.add("Q00", z90m)
    cnot.add("Q01", x90m)

cnot.plot()

In [ ]:
# Inverse CNOT sequence

y90 = x90.shifted(np.pi / 2)
z180 = qx.VirtualZ(np.pi)

xz90 = qx.pulse.CrossResonance(
    control_qubit="Q01",
    target_qubit="Q00",
    cr_amplitude=1.0,
    cr_duration=200,
    cr_ramptime=30,
    cr_phase=0.0,
    cancel_amplitude=0.01,
    cancel_phase=np.pi / 6,
    echo=True,
    pi_pulse=x180,
)

hadamard = qx.PulseArray([z180, y90])

with qx.PulseSchedule() as cnot_inv:
    cnot_inv.add("Q00", hadamard)
    cnot_inv.add("Q01", hadamard)
    cnot_inv.add("Q00-Q01", z180)
    cnot_inv.call(cnot)
    cnot_inv.add("Q00-Q01", z180)
    cnot_inv.add("Q00", hadamard)
    cnot_inv.add("Q01", hadamard)

cnot_inv.plot()

In [ ]:
# CNOT sequence with CPMG

with qx.PulseSchedule() as cnot_dd:
    cnot_dd.call(cnot_inv)
    tau = (cnot.duration - 2 * x180.duration) // 4
    cnot_dd.add("Q02", qx.pulse.CPMG(tau=tau, pi=x180))

cnot_dd.plot()

In [ ]:
# Bell sequence

with qx.PulseSchedule() as bell:
    bell.add("Q00", hadamard)
    bell.call(cnot_dd)

bell.plot(show_physical_pulse=False)

In [ ]:
with qx.PulseSchedule() as echo:
    echo.call(bell)
    echo.call(bell.reversed())

echo.plot(show_physical_pulse=False)

echo.plot(show_physical_pulse=True)